In [45]:
#Libraries

import numpy as np
import cv2
from scipy.spatial import distance
import statistics

In [46]:
#---------------------------------------------------------------------
#Display the image
def display_image(name,image):
    
    cv2.imshow(name,image)
    cv2.waitKey(0)
#---------------------------------------------------------------------

In [78]:
def image_processing(image):
    
    img = image
    img_copy = img.copy()
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #display_image("Grayscale",gray)
    
    _,thresh = cv2.threshold(gray,90,255,cv2.THRESH_BINARY)
    #display_image("Theshold",thresh)
    
    median = cv2.medianBlur(thresh,5)
    #display_image("Blur",median)
    
    canny = cv2.Canny(median, 200, 100)
    
    circles = cv2.HoughCircles(canny, cv2.HOUGH_GRADIENT, 1, 20, param1 = 200, param2 = 20, minRadius = 10, maxRadius = 0)
    
    if circles is not None:
        inner_circle = np.uint16(np.around(circles[0][0])).tolist()
        
        #cv2.circle(image, (inner_circle[0], inner_circle[1]), inner_circle[2], (0, 255, 0), 1)
        #display_image("Hough",image)
    
        white_image = np.full((img.shape[0], img.shape[1]), 255, dtype=np.uint8)
        cv2.circle(white_image, (inner_circle[0], inner_circle[1]), inner_circle[2]+8, (0, 0, 0), -1)
        #display_image('Mask',white_image)
        
        roi = cv2.bitwise_or(median,white_image)
        #display_image('roi',roi)
        
        canny_image = cv2.Canny(roi, 200, 100)
        #display_image("canny_roi",canny_image)
        
        contours, hierarchy = cv2.findContours(canny_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        detected_contour = cv2.drawContours(image.copy(), contours, -1, (0,255,0), 1)
        display_image("Contours",detected_contour)
    
        #center
        x = cv2.circle(image.copy(),(inner_circle[0], inner_circle[1]), 1, (255, 100, 0), 2)
        display_image("Center",x)
        
        pixelpoints = cv2.findNonZero(canny_image)
        #print(pixelpoints)
        
        distance_list = []  
        
        for i,point in enumerate(pixelpoints):
            
            contour_point = (point[0][0],point[0][1])
            center = (inner_circle[0],inner_circle[1])
            dist = distance.euclidean(contour_point,center)
            distance_list.append(dist)
        #print(distance_list)
        
        radius_list = []
        for i,val in enumerate(distance_list):
            
            radius = val*0.2645833333
            if radius > 7:
                continue
            else:
                radius_list.append(radius)
        #print(radius_list)
        
        
        final_value = statistics.median(radius_list)
        print('Radius is : ',final_value)
        
        print("Diameter is ",final_value * 2)
        

In [81]:
image = cv2.imread('eye.jpg',1)
display_image("Original",image)
pp = image_processing(image)

Radius is :  6.776639817096015
Diameter is  13.55327963419203
